## Initial Settings

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time
import os
import sys
import math
import random
import numpy as np
import cv2
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
from pynq import Overlay, MMIO, PL, allocate
from pynq.lib.video import *
from pynq_dpu import DpuOverlay
import spidev
import colorsys


# Load the overlay
overlay = DpuOverlay("../dpu/dpu.bit")

# # Load YOLOv3
overlay.load_model("../xmodel/top-tiny-yolov3_coco_256.xmodel")

## 모터 초기설정

In [6]:
# 모터 설정
motor_0_address = 0x00A0000000
motor_1_address = 0x00A0010000
motor_2_address = 0x00A0020000
motor_3_address = 0x00A0030000
motor_4_address = 0x00A0040000
motor_5_address = 0x00A0050000

address_range = 0x10000

motor_0 = MMIO(motor_0_address , address_range)
motor_1 = MMIO(motor_1_address , address_range) # CHANGE 
motor_2 = MMIO(motor_2_address , address_range)
motor_3 = MMIO(motor_3_address , address_range)
motor_4 = MMIO(motor_4_address , address_range) # CHANGE
motor_5 = MMIO(motor_5_address , address_range)

In [7]:
# 모터 기본 설정
# size = 2ms ==3.33ns x 600600.600..... 
# about 600600
size = 600600
motor_0_percent = 0/100
motor_1_percent = 80/100
motor_2_percent = 50/100
motor_3_percent = 0/100
motor_4_percent = 50/100
motor_5_percent = 80/100

motor_0_duty = size * motor_0_percent
motor_1_duty = size * motor_1_percent
motor_2_duty = size * motor_2_percent
motor_3_duty = size * motor_3_percent
motor_4_duty = size * motor_4_percent
motor_5_duty = size * motor_5_percent
print(int(motor_0_duty))


motor_0.write(0x00,size)           # size
motor_0.write(0x04,int(motor_0_duty))   # DUTY
motor_0.write(0x08,0)              # valid

motor_1.write(0x00,size)  # size
motor_1.write(0x04,int(motor_1_duty))   # DUTY
motor_1.write(0x08,0)       # valid

motor_2.write(0x00,size)  # size
motor_2.write(0x04,int(motor_2_duty))   # DUTY
motor_2.write(0x08,0)       # valid

motor_3.write(0x00,size)  # size
motor_3.write(0x04,int(motor_3_duty))   # DUTY
motor_3.write(0x08,0)       # valid

motor_4.write(0x00,size)  # size
motor_4.write(0x04,int(motor_4_duty))   # DUTY
motor_4.write(0x08,0)       # valid

motor_5.write(0x00,size)  # size
motor_5.write(0x04,int(motor_5_duty))   # DUTY
motor_5.write(0x08,0)       # valid

0


## MOTOR Control TEST

In [8]:
# motor 0 rear_right_front
motor_0.write(0x08,1)             
motor_1.write(0x08,0)              

motor_2.write(0x08,0)           
motor_3.write(0x08,0)         

motor_4.write(0x08,0)             
motor_5.write(0x08,0)             

In [9]:
# motor 1 steering_right
motor_0.write(0x08,0)             
motor_1.write(0x08,1)            

motor_2.write(0x08,0)              
motor_3.write(0x08,0)          

motor_4.write(0x08,0)              
motor_5.write(0x08,0)              

In [10]:
# motor 2 rear_left_back
motor_0.write(0x08,0)              
motor_1.write(0x08,0)          

motor_2.write(0x08,1)             
motor_3.write(0x08,0)             

motor_4.write(0x08,0)          
motor_5.write(0x08,0)        

In [11]:
# motor 3 rear_left_front
motor_0.write(0x08,0)            
motor_1.write(0x08,0)            

motor_2.write(0x08,0)           
motor_3.write(0x08,1)     

motor_4.write(0x08,0)           
motor_5.write(0x08,0)      

In [12]:
# motor 4  rear_right_back
motor_0.write(0x08,0)          
motor_1.write(0x08,0)         

motor_2.write(0x08,0)       
motor_3.write(0x08,0)            

motor_4.write(0x08,1)           
motor_5.write(0x08,0)           

In [13]:
# motor 5 steering_left 
motor_0.write(0x08,0)              
motor_1.write(0x08,0)           

motor_2.write(0x08,0)           
motor_3.write(0x08,0)           

motor_4.write(0x08,0)        
motor_5.write(0x08,1)      

In [14]:
# OFF
motor_5.write(0x08,0)              # valid  steering_left
motor_1.write(0x08,0)              # valid  steering_right
motor_2.write(0x08,0)              # valid rear_left_back
motor_3.write(0x08,0)              # valid rear_left_front
motor_4.write(0x08,0)              # valid rear_right_back
motor_0.write(0x08,0)              # valid rear_right_front

## SPI(Analog Read) Test

In [11]:
import spidev
import time
import sys

# SPI 설정
spi0 = spidev.SpiDev()

spi0.open(0, 0) 

# SPI 속도 설정 (최대 20MHz, Pmod AD1은 최대 1MHz 권장)
spi0.max_speed_hz = 20000000

# SPI 모드 설정 (모드 0)
spi0.mode = 0b00

def read_adc(spi):
    adc_response = spi.xfer2([0x00, 0x00])
    adc_value = ((adc_response[0] & 0x0F) << 8) | adc_response[1]
    return adc_value 

try:
    while True:
        # D0 채널에서 데이터 읽기
        adc_value0 = read_adc(spi0)
        # 출력 내용을 덮어쓰기
        #sys.stdout.write(f"\rD0 채널 값: {adc_value0*3.3/4096}")
        sys.stdout.write(f"\rD0 채널 값: {adc_value0}")
        sys.stdout.flush()
        time.sleep(0.5)
except KeyboardInterrupt:
    spi0.close()
#우측 250
#좌측 997

D0 채널 값: 9728

## YOLOv3 Utility functions


In [ ]:
anchor_list = [10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319]
anchors = np.array(anchor_list).reshape(-1, 2)
# 클래스 정보 가져오기 함수
def get_class(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names
    
classes_path = "../xmodel/lane_class.txt"
class_names = get_class(classes_path)

num_classes = len(class_names)
hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x: 
                  (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), 
                  colors))
random.seed(0)
random.shuffle(colors)
random.seed(None)

# 이미지 비율을 유지하며 padding하여 리사이즈하는 함수
def letterbox_image(image, size):
    ih, iw, _ = image.shape
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)
    image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_LINEAR)
    new_image = np.ones((h, w, 3), np.uint8) * 128
    h_start = (h-nh)//2
    w_start = (w-nw)//2
    new_image[h_start:h_start+nh, w_start:w_start+nw, :] = image
    return new_image

# 이미지 전처리 함수
def pre_process(image, model_image_size):
    image = image[..., ::-1]
    image_h, image_w, _ = image.shape
    if model_image_size != (None, None):
        assert model_image_size[0] % 32 == 0, 'Multiples of 32 required'
        assert model_image_size[1] % 32 == 0, 'Multiples of 32 required'
        boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    else:
        new_image_size = (image_w - (image_w % 32), image_h - (image_h % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0) 	
    return image_data

def _get_feats(feats, anchors, num_classes, input_shape):
    num_anchors = len(anchors)
    anchors_tensor = np.reshape(np.array(anchors, dtype=np.float32), [1, 1, 1, num_anchors, 2])
    grid_size = np.shape(feats)[1:3]
    nu = num_classes + 5
    predictions = np.reshape(feats, [-1, grid_size[0], grid_size[1], num_anchors, nu])
    grid_y = np.tile(np.reshape(np.arange(grid_size[0]), [-1, 1, 1, 1]), [1, grid_size[1], 1, 1])
    grid_x = np.tile(np.reshape(np.arange(grid_size[1]), [1, -1, 1, 1]), [grid_size[0], 1, 1, 1])
    grid = np.concatenate([grid_x, grid_y], axis=-1)
    grid = np.array(grid, dtype=np.float32)

    # Confidence score 계산 확인
    box_xy = (1 / (1 + np.exp(-predictions[..., :2])) + grid) / np.array(grid_size[::-1], dtype=np.float32)
    box_wh = np.exp(predictions[..., 2:4]) * anchors_tensor / np.array(input_shape[::-1], dtype=np.float32)
    box_confidence = 1 / (1 + np.exp(-predictions[..., 4:5]))
    box_class_probs = 1 / (1 + np.exp(-predictions[..., 5:]))

    return box_xy, box_wh, box_confidence, box_class_probs



def correct_boxes(box_xy, box_wh, input_shape, image_shape):
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = np.array(input_shape, dtype = np.float32)
    image_shape = np.array(image_shape, dtype = np.float32)
    new_shape = np.around(image_shape * np.min(input_shape / image_shape))
    offset = (input_shape - new_shape) / 2. / input_shape
    scale = input_shape / new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes = np.concatenate([
        box_mins[..., 0:1],
        box_mins[..., 1:2],
        box_maxes[..., 0:1],
        box_maxes[..., 1:2]
    ], axis = -1)
    boxes *= np.concatenate([image_shape, image_shape], axis = -1)
    return boxes


def boxes_and_scores(feats, anchors, classes_num, input_shape, image_shape):
    box_xy, box_wh, box_confidence, box_class_probs = _get_feats(feats, anchors, classes_num, input_shape)
    boxes = correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = np.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = np.reshape(box_scores, [-1, classes_num])
    return boxes, box_scores

def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2-x1+1)*(y2-y1+1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= 0.1)[0]  # threshold
        order = order[inds + 1]

    return keep

# 모델 평가 함수: YOLOv3-Tiny의 2-출력 레이어 구조에 맞게 anchor_mask 설정
def evaluate(yolo_outputs, image_shape, class_names, anchors, max_boxes=20):
    score_thresh = 0.3
    anchor_mask = [[3, 4, 5], [0, 1, 2]]  # YOLOv3-Tiny에 맞춘 2개의 출력 레이어용 anchor mask
    boxes = []
    box_scores = []
    input_shape = np.shape(yolo_outputs[0])[1:3] * np.array([32, 32])

    for i in range(len(yolo_outputs)):
        _boxes, _box_scores = boxes_and_scores(
            yolo_outputs[i], anchors[anchor_mask[i]], len(class_names), 
            input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = np.concatenate(boxes, axis=0)
    box_scores = np.concatenate(box_scores, axis=0)

    mask = box_scores >= score_thresh
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(len(class_names)):
        class_boxes_np = boxes[mask[:, c]]
        class_box_scores_np = box_scores[:, c][mask[:, c]]
        
        # Non-Max Suppression with max_boxes limit
        nms_index_np = nms_boxes(class_boxes_np, class_box_scores_np)
        nms_index_np = nms_index_np[:max_boxes]  # Limit to max_boxes
        
        class_boxes_np = class_boxes_np[nms_index_np]
        class_box_scores_np = class_box_scores_np[nms_index_np]
        classes_np = np.ones_like(class_box_scores_np, dtype=np.int32) * c
        boxes_.append(class_boxes_np)
        scores_.append(class_box_scores_np)
        classes_.append(classes_np)
        
    boxes_ = np.concatenate(boxes_, axis=0)
    scores_ = np.concatenate(scores_, axis=0)
    classes_ = np.concatenate(classes_, axis=0)

    # confidence scores 출력
    print("Confidence scores for detected boxes:", scores_)
    
    return boxes_, scores_, classes_


## DPU Setting

In [16]:
dpu = overlay.runner
inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()
shapeIn = tuple(inputTensors[0].dims)
shapeOut0 = (tuple(outputTensors[0].dims)) # (1, 13, 13, 75)
shapeOut1 = (tuple(outputTensors[1].dims)) # (1, 26, 26, 75)
outputSize0 = int(outputTensors[0].get_data_size() / shapeIn[0]) # 12675
outputSize1 = int(outputTensors[1].get_data_size() / shapeIn[0]) # 50700
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
output_data = [np.empty(shapeOut0, dtype=np.float32, order="C"), 
               np.empty(shapeOut1, dtype=np.float32, order="C")]
image = input_data[0]

## Driving Funtions

In [ ]:
# SPI 초기화
spi0 = spidev.SpiDev()
spi0.open(0, 0)
spi0.max_speed_hz = 20000000
spi0.mode = 0b00


class RobotController:
    def __init__(self):
        # 이미지 처리 객체
        self.image_processor = ImageProcessor()

        # ----------------------
        # 모터 설정
        # ----------------------
        self.size = 600600         # size = 2ms ==3.33ns x 600600.600.....
        self.speed = 50             # 주행 속도(0 ~ 100)
        self.steering_speed = 50   # 조향 속도(0 ~ 100)

        # 모터들을 딕셔너리로 관리
        self.motors = {
            'motor_0': motor_0,  # right front
            'motor_1': motor_1,  # steering right
            'motor_2': motor_2,  # left back
            'motor_3': motor_3,  # left front
            'motor_4': motor_4,  # right back
            'motor_5': motor_5   # steering left
        }

        self.init_motors()

        # ----------------------
        # 가변저항 범위 설정(ADC)
        # ----------------------
        self.resistance_most_left = 997
        self.resistance_most_right = 250

    def map_value(self, x, in_min, in_max, out_min, out_max):
        """
        x를 in_min~in_max 범위에서 out_min~out_max 범위로 매핑
        """
        return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

    def init_motors(self):
        """
        모터 초기화:
        - size 레지스터 설정
        - 초기 duty(0%) 설정
        - valid(0x08)는 우선 0으로 설정
        """
        for key, motor in self.motors.items():
            motor.write(0x00, self.size)   # size 설정
            duty = int(self.size * (self.speed / 100))
            motor.write(0x04, duty)        # 초기 duty = 0%
            motor.write(0x08, 0)          # valid = 0 (비활성)

    # ----------------------
    # 조향(steering) 제어부
    # ----------------------
    def right(self):
        """
        우회전 제어
        - steering_left(motor_5) disable
        - steering_right(motor_1) enable
        - duty는 steering_speed 비율
        """
        duty = int(self.size * (self.steering_speed / 100))
        self.motors['motor_5'].write(0x08, 0)  # disable steering_left
        self.motors['motor_1'].write(0x08, 1)  # enable steering_right
        self.motors['motor_1'].write(0x04, duty)

    def left(self):
        """
        좌회전 제어
        - steering_left(motor_5) enable
        - steering_right(motor_1) disable
        """
        duty = int(self.size * (self.steering_speed / 100))
        self.motors['motor_5'].write(0x08, 1)  # enable steering_left
        self.motors['motor_1'].write(0x08, 0)  # disable steering_right
        self.motors['motor_5'].write(0x04, duty)

    def stay(self):
        """
        중립 상태(조향 없음)
        - 좌우 조향 모터 모두 disable
        - duty는 기본값(steering_speed)로 설정
        """
        duty = int(self.size * (self.steering_speed / 100))
        self.motors['motor_5'].write(0x08, 0)
        self.motors['motor_1'].write(0x08, 0)
        self.motors['motor_5'].write(0x04, duty)
        self.motors['motor_1'].write(0x04, duty)

    # ----------------------
    # 주행(드라이브) 제어부
    # ----------------------
    def set_left_speed(self, speed):
        """
        왼쪽 모터(2: back, 3: front) 속도 설정
        - speed > 0 이면 전진
        - speed < 0 이면 후진
        - speed = 0 이면 duty=0
        """
        duty = int(self.size * (abs(speed) / 100))
        self.motors['motor_2'].write(0x04, duty)  # left back
        self.motors['motor_3'].write(0x04, duty)  # left front

        # valid 설정: 0 -> 전진(enable front), 1 -> 후진(enable back)
        if speed > 0:
            # 전진
            self.motors['motor_2'].write(0x08, 0)  # left back disable
            self.motors['motor_3'].write(0x08, 1)  # left front enable
        elif speed < 0:
            # 후진
            self.motors['motor_2'].write(0x08, 1)  # left back enable
            self.motors['motor_3'].write(0x08, 0)  # left front disable
        else:
            # 속도가 0이면 valid도 0으로 하여 motor off
            self.motors['motor_2'].write(0x08, 0)
            self.motors['motor_3'].write(0x08, 0)

    def set_right_speed(self, speed):
        """
        오른쪽 모터(4: back, 0: front) 속도 설정
        - speed > 0 이면 전진
        - speed < 0 이면 후진
        - speed = 0 이면 duty=0
        """
        duty = int(self.size * (abs(speed) / 100))
        self.motors['motor_4'].write(0x04, duty)  # right back
        self.motors['motor_0'].write(0x04, duty)  # right front

        # valid 설정: 0 -> 전진(enable front), 1 -> 후진(enable back)
        if speed > 0:
            # 전진
            self.motors['motor_4'].write(0x08, 0)  # right back disable
            self.motors['motor_0'].write(0x08, 1)  # right front enable
        elif speed < 0:
            # 후진
            self.motors['motor_4'].write(0x08, 1)  # right back enable
            self.motors['motor_0'].write(0x08, 0)  # right front disable
        else:
            # 속도가 0이면 valid도 0으로 하여 motor off
            self.motors['motor_4'].write(0x08, 0)
            self.motors['motor_0'].write(0x08, 0)

    # ----------------------
    # ADC 입력(가변저항) 처리
    # ----------------------
    def read_adc(self, spi):
        """
        SPI를 통해 12비트 ADC 값을 읽어 반환
        """
        adc_response = spi.xfer2([0x00, 0x00])
        # 12비트 중 하위 8비트 + 상위 4비트 처리
        adc_value = ((adc_response[0] & 0x0F) << 8) | adc_response[1]
        return adc_value

    # ----------------------
    # 기울기(각도) 매핑 
    # 학생분들이 주로 수정할 Main Logic
    # ----------------------
    def map_angle_to_range(self, angle):
        """
        검출된 angle(기울기)을 조향에 사용할 범위
        - 기울기 값이 0보다 크면 7을 반환
        - 기울기 값이 0보다 작다면 -7을 반환
        """        
        if angle > 0:
            return 7
        elif angle < 0:
            return -7
        return 0

    # ----------------------
    # 메인 제어 로직
    # ----------------------
    def control_motors(self, angle):
        """
        - ADC로부터 얻은 현재 조향 위치(mapped_resistance)와
          영상처리로 구한 목표 각도(mapped_angle)를 비교
        - 비교 결과에 따라 조향(좌, 우, stay)
        - 이후 후방 왼/오른쪽 모터에 self.speed 적용
        """
        # ADC 읽기
        adc_value0 = self.read_adc(spi0)
        mapped_resistance = self.map_value(adc_value0,
                                           self.resistance_most_right,
                                           self.resistance_most_left,
                                           -7, 7)

        mapped_angle = self.map_angle_to_range(angle)

        print("ADC:", adc_value0, 
              "/ mapped_resistance:", mapped_resistance, 
              "/ mapped_angle:", mapped_angle)

        # 허용 오차(tolerance) 범위 내에서 조향 유지
        tolerance = 0
        if abs(mapped_resistance - mapped_angle) <= tolerance:
            self.stay()
        elif mapped_resistance > mapped_angle:
            self.left()
        else:
            self.right()

        # 주행 모터 설정 (현재 self.speed 값 사용)
        self.set_left_speed(self.speed)
        self.set_right_speed(self.speed)

    def process_and_control(self, frame):
        """
        1) frame을 ImageProcessor를 통해 처리하여 기울기(각도) 계산
        2) 해당 각도(angle)에 맞춰 모터 제어
        3) 처리된 이미지를 반환(표시용)
        """
        slope, processed_image = self.image_processor.process_frame(frame)
        self.control_motors(slope)
        return processed_image

    def stop_motors(self):
        """
        모든 모터 정지(유효신호 0, duty 0)
        """
        for key, motor in self.motors.items():
            motor.write(0x04, 0)   # duty=0
            motor.write(0x08, 0)   # valid=0

    def run(self, video_path=None, camera_index=0):
        """
        - video_path가 주어지면 해당 영상 파일로부터 프레임 읽어오기
        - 아니면 기본 카메라 인덱스(0 등)로부터 실시간 영상 받아오기
        - 각 프레임마다 process_and_control 수행
        """
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            cap = cv2.VideoCapture(camera_index)
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        if not cap.isOpened():
            print("카메라(또는 영상 파일)를 열 수 없습니다.")
            return

        image_widget = widgets.Image(format='jpeg')

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("프레임을 읽을 수 없습니다. 영상이 끝났거나 오류가 발생했습니다.")
                    break

                processed_image = self.process_and_control(frame)
                # 처리된 이미지 표시 (노트북 상에서)
                ret_jpeg, jpeg = cv2.imencode('.jpeg', processed_image)
                if ret_jpeg:
                    image_widget.value = jpeg.tobytes()
                    display(image_widget)
                    clear_output(wait=True)
                else:
                    print("이미지 인코딩 실패")

        except KeyboardInterrupt:
            print("사용자에 의해 중지되었습니다.")
        finally:
            cap.release()
            self.stop_motors()  # 모든 모터 정지
            print("모터 정지 및 자원 해제 완료")


print("done")


done


## ImageProcessor

In [ ]:
class ImageProcessor:
    def __init__(self):
        """
        ImageProcessor 초기화: 차선 중심 계산 시 y 좌표(높이)
        - point_detection_height
        """
        self.point_detection_height = 20  # 차선 중심 계산 시 y 좌표(높이)

    def roi_rectangle_below(self, img, cutting_idx):
        """
        이미지 하부를 잘라서 ROI 추출
        :param img: 원본 이미지
        :param cutting_idx: 위에서부터 얼마나 자를지(픽셀 인덱스)
        :return: 잘려진 ROI 이미지
        """
        return img[cutting_idx:, :]

    def warpping(self, image, srcmat, dstmat):
        """
        원근 변환(Perspective Transform) 수행
        :param image: 원본 이미지
        :param srcmat: 원근 변환 전 좌표
        :param dstmat: 원근 변환 후 좌표
        :return: 변환된 이미지, 역변환 행렬
        """
        (h, w) = (image.shape[0], image.shape[1])
        transform_matrix = cv2.getPerspectiveTransform(srcmat, dstmat)
        minv = cv2.getPerspectiveTransform(dstmat, srcmat)  # 역변환용
        warped = cv2.warpPerspective(image, transform_matrix, (w, h))
        return warped, minv

    def bird_convert(self, img, srcmat, dstmat):
        """
        주어진 srcmat, dstmat 좌표를 이용하여 버드아이 뷰(bird's-eye view)로 변환
        :param img: 원본 이미지
        :param srcmat: 원근 변환 전 좌표
        :param dstmat: 원근 변환 후 좌표 
        :return: 변환된 이미지
        """
        srcmat = np.float32(srcmat)
        dstmat = np.float32(dstmat)
        img_warped, minverse = self.warpping(img, srcmat, dstmat)
        return img_warped

    def calculate_angle(self, x1, y1, x2, y2):
        """
        점 (x1, y1)과 (x2, y2)를 연결하는 직선의 기울기를 각도로 계산
        :return: 라인의 각도(degree), 수평 기준
        """
        if x1 == x2:
            # 수직선인 경우, 각도 90도로 처리
            return 90.0
        slope = (y2 - y1) / (x2 - x1)
        angle = math.degrees(math.atan(slope))
        # 좌표계가 반대로 되어 있어, 디버깅을 쉽게하기 위해, 기울기 값에 -를 붙인 뒤 반환
        return -angle

    def detect_lane_center_x(self, xyxy_results):
        """
        입력된 바운딩 박스들(xyxy_results) 중, 가장 오른쪽 레이블의 중심 X를 찾음.
        :param xyxy_results: [ [y1, x1, y2, x2], ... ] 형식
        :return: 가장 오른쪽 레이블(차선)의 중심 X 좌표. 없으면 None
        """
        rightmost_lane_x_min = None
        rightmost_lane_x_max = None
        rightmost_x_position = -float('inf')  # 최대값 비교를 위해 매우 작은 값으로 초기화

        for box in xyxy_results:
            y1, x1, y2, x2 = box  # 상단좌측(y1,x1), 하단우측(y2,x2)
            # x1이 클수록 더 오른쪽에 있는 레이블이라고 가정
            if x1 > rightmost_x_position:
                rightmost_x_position = x1
                rightmost_lane_x_min = int(x1)
                rightmost_lane_x_max = int(x2)

        # 가장 오른쪽 레이블을 찾은 경우 중심 X 계산
        if rightmost_lane_x_min is not None and rightmost_lane_x_max is not None:
            lane_center_x = (rightmost_lane_x_min + rightmost_lane_x_max) // 2
            return lane_center_x
        else:
            # 찾지 못한 경우
            return None

    def process_frame(self, img):
        """
        주된 처리 흐름:
        1) BEV
        2) ROI 추출
        3) DPU 추론
        4) YOLOv3 output debugging
        5) 가장 오른쪽 레인(center) X 좌표 구하기
        6) 각도(Angle) 계산 + 시각화(라인/원 그리기)
        :param img: 원본 프레임
        :return: (right_lane_angle, 디버깅용 이미지)
        """

        # 원본 이미지 크기
        (h, w) = (img.shape[0], img.shape[1])

        # 버드아이 뷰 변환을 위한 dst, src 예시 좌표 (실 환경에 맞게 조정)
        dst_mat = [
            [round(w * 0.3), 0],
            [round(w * 0.7), 0],
            [round(w * 0.7), h],
            [round(w * 0.3), h]
        ]
        src_mat = [
            [238, 316],
            [402, 313],
            [501, 476],
            [155, 476]
        ]

        # (1) BEV 변환
        bird_img = self.bird_convert(img, srcmat=src_mat, dstmat=dst_mat)

        # (2) ROI(아랫부분만) 추출 (예: 300픽셀 지점부터 아래)
        roi_image = self.roi_rectangle_below(bird_img, cutting_idx=300)

        # (3) DPU 입력 크기(256,256)에 맞춰 리사이즈
        resized_img = cv2.resize(roi_image, (256, 256))


        # 전처리
        image_size = resized_img.shape[:2]  # (256, 256)
        image_data = np.array(pre_process(resized_img, (256, 256)), dtype=np.float32)
        
        start_time = time.time()  # 추론 시작시간
        image[0, ...] = image_data.reshape(shapeIn[1:])  # DPU input에 데이터 복사
        job_id = dpu.execute_async(input_data, output_data)
        dpu.wait(job_id)
        end_time = time.time()    # 추론 종료시간

        # DPU output
        conv_out0 = np.reshape(output_data[0], shapeOut0)
        conv_out1 = np.reshape(output_data[1], shapeOut1)
        yolo_outputs = [conv_out0, conv_out1]

        # YOLO 디코딩 (boxes, scores, classes)
        boxes, scores, classes = evaluate(yolo_outputs, image_size, class_names, anchors)

        # (4) 바운딩 박스 그리기
        for i, box in enumerate(boxes):
            top_left = (int(box[1]), int(box[0]))  # (x_min, y_min)
            bottom_right = (int(box[3]), int(box[2]))  # (x_max, y_max)
            color = (0, 255, 0)  # 바운딩 박스 색상
            cv2.rectangle(resized_img, top_left, bottom_right, color, 2)

        # (5) BBOX의 중심 X 구하기
        right_lane_center = self.detect_lane_center_x(boxes)
        if right_lane_center is None:
            # 레인을 찾지 못할 경우 임의로 각도 60도 (예외처리)
            print("차선 중심을 찾을 수 없습니다.")
            right_lane_angle = 60
            return right_lane_angle, resized_img

        # (6) 각도 계산
        #  - 기준점 (200, 240) vs 검출점 (right_lane_center(바운딩박스 중심 좌표), self.point_detection_height(초기 설정한 y값))
        right_lane_angle = self.calculate_angle(
            x1=200, y1=240,
            x2=right_lane_center, y2=self.point_detection_height
        )

        # 시각화
        start_point = (200, 240)
        end_point = (right_lane_center, self.point_detection_height)

        cv2.line(resized_img, start_point, end_point, (0, 0, 255), 3)

        cv2.circle(resized_img, (right_lane_center, self.point_detection_height), 8, (0, 255, 255), -1)

        text_angle = f"Angle: {right_lane_angle:.2f} deg"
        cv2.putText(
            resized_img, text_angle,
            (right_lane_center - 150, self.point_detection_height),  # 라인 끝에서 약간 오른쪽으로
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2
        )

        # (7) 실행 시간, FPS 계산 및 출력
        execution_time_ms = (end_time - start_time) * 1000
        fps = 1000 / execution_time_ms if execution_time_ms > 0 else float('inf')
        print(f"Execution Time: {execution_time_ms:.2f} ms, FPS: {fps:.2f}")
        print("right_lane_angle:", right_lane_angle)
        print("bbox height:", self.point_detection_height)

        return right_lane_angle, resized_img


print("done")


done


## Driving!

In [ ]:
controller = RobotController()
# 카메라 im
controller.run(camera_index=0)

# 비디오 영상 in
#controller.run(video_path="../test_video/test_video.mp4")

Confidence scores for detected boxes: [0.9524826]
Execution Time: 12.19 ms, FPS: 82.02
right_lane_angle -68.5795408807901
mapped_resistance -18.018741633199472
mapped_angle -15
right
중지되었습니다.
